In [1]:
import pandas as pd
import wrangle
import new_wrangle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

# acquire data

Basic acquire data, we are not modifying anything. We just wan to see the data .

In [2]:
# use function to acquire data
df1= wrangle.acquire_data()

In [3]:
#check  info
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6435 entries, 0 to 6434
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
 8   Type          6435 non-null   object 
 9   Size          6435 non-null   int64  
dtypes: float64(5), int64(3), object(2)
memory usage: 553.0+ KB


In [4]:
df1.shape

(6435, 10)

**Takeaways**
- we have 10 columns and 6435 columns
- no nulls
- we nned to change some columns type

# Ouliers store_size

We are exploring the store_type vs store_size, we notice that there are outliers

In [ ]:
sns.boxplot(x= df1['Type'], y= df1['Size'])

In [ ]:
#check only the outliers
sns.boxplot(x= df1['Type'], y= df1['Size'])
plt.ylim(0,46000)

we can see there are outliers for B and A and we decided to change those to  store type C

In [ ]:
#getting a df that has all store_size < 50000 and df.store_type != "C"
df3 = df1 [(df1.Size < 50000) & (df1.Type != "C")]

In [ ]:
#lets see which stores type A  are df.store_size < 50000
df3[df3.Type == "A"].groupby('Store').count()

In [ ]:
#lets see which stores type A  are df.store_size < 50000
df3[df3.Type == "B"].groupby('Store').count()

**Takeaways**
- Instead of removing theses outliers, we decided to change them to the type based on the size.
- I gave the store_id to Natasha so she is going to change the type in the wrangle function.


# Wrangle

the wrangle_walmar function is renaming columns, adding new columns, dummi variables, and set date as index (datetime type)

In [ ]:
#using the funcion wrangle
df= new_wrangle.wrangle_walmart()

In [ ]:
#chec the data
df.info()

In [ ]:
df.head(1)

In [ ]:
#checking if the store_type has no outliers
sns.boxplot(x= df['store_type'], y= df['store_size'])

In [ ]:
def distribution (df):
    '''
    takes in a df and plot individual variable distributions excluding object type
    '''
    plt.figure
    plt.style.use("ggplot")
    cols =df.columns.to_list()
    for col in cols:
        if df[col].dtype != 'object':
            plt.hist(df[col],color ='blue')
            plt.title(f'Distribution of {col}')
            plt.xlabel(col)
            plt.ylabel('Number of Weeks ')
            plt.xticks(rotation=45)
            plt.ticklabel_format(style = 'plain')
            
            plt.show()

In [ ]:
distribution(df)

In [ ]:
df.groupby('store_type').mean()

In [ ]:
#let see distribution by store type
store_type = ["A", "B", "C"]
for store in store_type:
    print("Store type: ", store)
    distribution(df[df.store_type == store])
    

In [ ]:
#how many stores are Type C
len(df[df.store_type == 'C'].store_id.value_counts())

In [ ]:
# storeid for type c
df[df.store_type == 'C'].store_size.value_counts()

In [ ]:
#store if for type B
df[df.store_type == 'B'].store_size.value_counts().sort_index()

In [ ]:
#how many stores are Type B
len(df[df.store_type == 'B'].store_id.value_counts().sort_index())

In [ ]:
#store_Id for type A
df[df.store_type == 'A'].store_size.value_counts().sort_index()

In [ ]:
##how many stores are Type A
len(df[df.store_type == 'A'].store_id.value_counts().sort_index())

In [ ]:
df.info()

In [ ]:
df.shape

**Takeaways**
- after wrangle_walmar we ended with 29 columns, and 6435 observations

<hr style="border:2px solid blue"> </hr>

## the following steps were done to complete the wrangle function

# dummies 

we are going to create dummies for :
- Holiday_name


In [ ]:
#create dummy variables for 5 columns
def create_multiple_dummies (df, dumm_col = ['holiday_name', 'season', 'store_type', 'month', 'year']):
    '''
    Takes in a df and columns to create dummies.
    retunr the original df with de new columns (dimmies)
    '''
    #the column year is an integer we need to conver as string
    df['year']= df['year'].astype('string')
    #create dummy variables 
    for col in dumm_col:
        #create dummies
        df_dummies = pd.get_dummies(df[col], dummy_na=False)
        #  concat df_dummies with my df
        df = pd.concat([df, df_dummies], axis =1)
    #drop no_holiday columns and year
    df = df.drop(columns = ['no_holiday', 'year'])
    return df

In [ ]:
def create_dummies (df, dumm_col = ['holiday_name']):
    '''
    Takes in a df and columns to create dummies.
    retunr the original df with de new columns (dummies)
    '''
    #create dummy variables 
    for col in dumm_col:
        #create dummies
        df_dummies = pd.get_dummies(df[col], dummy_na=False)
        #  concat df_dummies with my df
        df = pd.concat([df, df_dummies], axis =1)
    #drop no_holiday columns and year
    df = df.drop(columns = ['no_holiday'])
    return df

In [ ]:
df['holiday_name'].value_counts()

In [ ]:
#test the function before I add it to .py file
df2 = create_dummies(df)

In [ ]:
df2.info()

In [ ]:
df2.columns

# Create new columns

#we need to create new columns  pre- christmas  and tax_season because on exploration(Alberto and Heather ) they found a peak in those dates. 

In [ ]:
#checking the previous weeks for christmas

In [ ]:
df2.head(1)

create a new column for is pre_christmas

-Christmas:

    - 31-Dec-10,  (pre_christmas = 24-Dec-10, 17-Dec 10)
    - 30-Dec-11, (pre_christmas = 23-Dec-11, 16-Dec 11)

In [ ]:
# create new colum and add zeros to everything
df2 ['pre_christmas'] = 0

In [ ]:
df2.columns

In [ ]:
#getting the list for pre_christmas
pre_c= ['2010-12-24', '2010-12-17', '2011-12-23', '2011-12-16']

In [ ]:
#add value 1 for only pre_christmas weeks
#df2.loc[pre_c, 'pre_christmas'] = 1

In [ ]:
#CHECK THE VALUES
#df2['pre_christmas'].value_counts()

#### ADD TAX SEASON

###  first 2 weeks of April
- 2010-04-02 & 2010-04-09
- 2011-04-01 & 2011-04-08
- 2012-04-06 & 2012-04-13

In [ ]:
#crete a new column and assign 0 as value
df2['tax_season'] = 0 

In [ ]:
#getting the list for tax
tax= ['2010-04-02 ', '2010-04-09', '2011-04-01', '2011-04-08', '2012-04-06', '2012-04-13']

In [ ]:
#add value 1 for only for the list above
df2.loc[tax, 'tax_season'] = 1

In [ ]:
#CHECK THE VALUES
df2['tax_season'].value_counts()

# Scale

In [ ]:
df2.head(1)

**note :** before scaling we need to split our data


In [ ]:
#select only the numeric columns 
num_df = df2.select_dtypes(exclude='object')

In [ ]:
num_df.shape

In [ ]:
num_df.head()

In [ ]:
#split using a function
train, test, X_train, y_train, X_test, y_test = new_wrangle.train_test(num_df, 'weekly_sales')

In [ ]:
# imports to scale
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

In [ ]:
def scaled_df ( train_df , test_df, columns,  scaler):
    '''
    Take in a 3 df and a type of scaler that you  want to  use. it will scale all columns
    except object type. Fit a scaler only in train and tramnsform in train, validate and test.
    returns  new dfs with the scaled columns.
    scaler : MinMaxScaler() or RobustScaler(), StandardScaler() 
    Example:
    scaled_df( X_train , X_test, columns , RobustScaler())
    
    '''
    #import
    from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
    # fit our scaler
    scaler.fit(train_df[columns])
    # get our scaled arrays
    train_scaled = scaler.transform(train_df[columns])
    test_scaled= scaler.transform(test_df[columns])

    # convert arrays to dataframes
    train_scaled_df = pd.DataFrame(train_scaled, columns=columns).set_index([train_df.index.values])
    test_scaled_df = pd.DataFrame(test_scaled, columns=columns).set_index([test_df.index.values])

#     #add the columns that are not scaled
#     train_scaled_df = pd.concat([train_scaled_df, train_df.drop(columns = columns) ], axis= 1 )
#     test_scaled_df = pd.concat([test_scaled_df, test_df.drop(columns = columns) ], axis= 1 )
    #plot
    for col in columns: 
        plt.figure(figsize=(13, 6))
        plt.subplot(121)
        plt.hist(train_df[col], ec='black')
        plt.title('Original')
        plt.xlabel(col)
        plt.ylabel("counts")
        plt.subplot(122)
        plt.hist(train_scaled_df[col],  ec='black')
        plt.title('Scaled')
        plt.xlabel(col)
        plt.ylabel("counts")



    return train_scaled_df,  test_scaled_df

In [ ]:
columns =  X_train.columns.to_list()

In [ ]:
#test the function
#train_Xscaled_df,  test_Xscaled_df= scaled_df( X_train , X_test, columns , MinMaxScaler())

In [ ]:
def train_test(df, target):
    '''
    This function brings in the dataframe and the target feature
    then returns X_train, y_train, X_test and y_test with their respective shapes
    '''
    train = df[:'05-2012'] # includes everything until june 2016
    test = df['06-2012':"2012"] #includes last 6 months

    # split train into X (dataframe, drop target) & y (series, keep target only)
    X_train = train.drop(columns=[target])
    y_train = train[target]

    # split test into X (dataframe, drop target) & y (series, keep target only)
    X_test = test.drop(columns=[target])
    y_test = test[target]

    # Have function print datasets shape
    print(f'X_train -> {X_train.shape}')
    print(f'X_test -> {X_test.shape}')

    return train, X_train, y_train, X_test, y_test

In [ ]:
def split_scale and scale (df, target, scaler):
    '''
    takes in a df and creates dummy variables, select only the numeric columns and  split into X_train, y_train, 
    X_test, y_test and scaled X_train, X_test.
    return   X_train_scaled, y_train_scaled, X_test, y_test
    '''



    #split
    train, X_train, y_train, X_test, y_test = train_test(num_df, target)

    #select the columns to scale
    columns =  X_train.select_dtypes(exclude='object').columns.to_list()
    #scale 
    X_train_scaled, X_test_scaled = scaled_df( X_train , X_test, columns , scaler)

    return train, X_train_scaled, X_test_scaled, y_train, y_test
        
        
        
        
    

In [ ]:
#split
train, test, X_train, y_train, X_test, y_test = new_wrangle.train_test(df, 'weekly_sales')

In [ ]:
#select the columns to scale
col =  X_train.select_dtypes(exclude='object').columns.to_list()

In [ ]:
col

In [ ]:
#scale 
X_train_scaled, X_test_scaled = scaled_df( X_train , X_test, col , MinMaxScaler())

In [ ]:
X_train_scaled.info()

In [ ]:
train, X_train_scaled, y_train_scaled, X_test, y_test = prepare_modeling_dummies (df, 'weekly_sales', MinMaxScaler())

In [ ]:
train.shape

In [ ]:
X_train_scaled.shape

# split 

In [ ]:
train, test,  X_train_scaled, X_test_scaled, y_train, y_test = new_wrangle.split_scale(df, 'weekly_sales', MinMaxScaler())

In [ ]:
X_train_scaled.info()

____________________

# Wrangle and split

let's test the functions wrangle and split and scale

In [5]:
df =new_wrangle.wrangle_walmart()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6435 entries, 2010-02-05 to 2012-10-26
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   store_id            6435 non-null   object 
 1   weekly_sales        6435 non-null   float64
 2   holiday_flag        6435 non-null   int64  
 3   temperature         6435 non-null   int64  
 4   fuel_price          6435 non-null   float64
 5   inflation           6435 non-null   float64
 6   unemployment        6435 non-null   float64
 7   store_type          6435 non-null   object 
 8   store_size          6435 non-null   int64  
 9   month               6435 non-null   object 
 10  year                6435 non-null   int64  
 11  quarter             6435 non-null   int64  
 12  weekday             6435 non-null   object 
 13  week_of_year        6435 non-null   int64  
 14  deflated_series     6435 non-null   float64
 15  sales_delta_weekly  6435 non-null   f

**Note:**  we have nulls in our df because we created a new column last_year_sales that's why the fisrt year has nulls because we dond have information about the previous year

In [7]:
df.shape

(6435, 29)

In [8]:
df.holiday_name.value_counts()

no_holiday       5535
tax_season        270
pre_christmas     180
labor_day         135
super_bowl        135
thanksgiving       90
christmas          90
Name: holiday_name, dtype: int64

In [9]:
#drop nulls
df2= df.dropna()

In [10]:
df2.shape

(4095, 29)

In [11]:
train, test,  X_train_scaled, X_test_scaled, y_train, y_test = new_wrangle.split_scale(df2, 'weekly_sales', MinMaxScaler())

train -> (2866, 30)
test -> (1229, 30)


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2866 entries, 2011-07-08_store_14 to 2012-08-10_store_32
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                2866 non-null   object 
 1   store_id            2866 non-null   object 
 2   weekly_sales        2866 non-null   float64
 3   holiday_flag        2866 non-null   int64  
 4   temperature         2866 non-null   int64  
 5   fuel_price          2866 non-null   float64
 6   inflation           2866 non-null   float64
 7   unemployment        2866 non-null   float64
 8   store_type          2866 non-null   object 
 9   store_size          2866 non-null   int64  
 10  month               2866 non-null   object 
 11  year                2866 non-null   int64  
 12  quarter             2866 non-null   int64  
 13  weekday             2866 non-null   object 
 14  week_of_year        2866 non-null   int64  
 15  deflated_series     2866 no

In [13]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2866 entries, 2011-07-08_store_14 to 2012-08-10_store_32
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   holiday_flag        2866 non-null   float64
 1   temperature         2866 non-null   float64
 2   fuel_price          2866 non-null   float64
 3   inflation           2866 non-null   float64
 4   unemployment        2866 non-null   float64
 5   store_size          2866 non-null   float64
 6   year                2866 non-null   float64
 7   quarter             2866 non-null   float64
 8   week_of_year        2866 non-null   float64
 9   deflated_series     2866 non-null   float64
 10  sales_delta_weekly  2866 non-null   float64
 11  sales_delta_yearly  2866 non-null   float64
 12  gas_delta_weekly    2866 non-null   float64
 13  gas_delta_yearly    2866 non-null   float64
 14  last_year_sales     2866 non-null   float64
 15  last_week_sales     2866 no

In [ ]:
y_train

# Explore 

In [ ]:
df.shape

In [ ]:
train.shape

In [ ]:
train.head(1)

In [ ]:
train.info()

In [ ]:
#explore our target
plt.figure(figsize=(8, 10))
sns.histplot(train,
                 x=train['weekly_sales'],
                 #hue='loan_status',
                 multiple='layer'
                 )
plt.title("Distribution of Loan Status")
plt.xlabel('Weekly sales in dollars')
plt.ylabel('Number of weeks')
plt.xticks(rotation=45)
plt.ticklabel_format(style = 'plain')

plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(train['weekly_sales'])
plt.xticks(rotation=45)
#plt.ticklabel_format(style = 'plain')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(train['weekly_sales'])
plt.xlim(2500000)
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(train.corr(), cmap='coolwarm', annot=True, linewidth=0.5)
plt.show()

In [ ]:
#check the relation of my target with all the featues

In [ ]:
#let's focus on my target
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(train.corr()[['weekly_sales']].sort_values(by='weekly_sales', ascending=False), vmin=-3, vmax=3, annot=True, cmap='coolwarm')

**takeaways**
- last week_sales, last_year_sales and store_size have stroger correlation with out target
- pre_christmas has also a positive correlation
- we can not use deflated_series because was calculated using our target
- negative correlation unemployment 

## Exploration target vs Fearture

### -  weekly_sales vs store_size

In [ ]:
plt.figure(figsize=(16, 10))
sns.jointplot(x= 'weekly_sales', y= 'store_size', data = train, hue ='season')

**takeaways**
- the small the store, less weekly sales
- winter has the greater weekly sales

### -  weekly_sales vs temperature

In [ ]:
plt.figure(figsize=(12, 10))
sns.jointplot(x= 'weekly_sales', y= 'temperature', data = train, hue= "season")
plt.show()

**Takeaways**
- Winter and fall have the greater weekly_sales. may it is because of christmas and blackfriday

In [ ]:
# winter_df = train[train.season == "Winter"][['weekly_sales', 'temperature', 'season']]
# winter_df.head()

In [ ]:
# sns.jointplot(x= 'weekly_sales', y= 'temperature', data = winter_df, hue= "season")

In [ ]:
train.season.unique()

In [ ]:
color_list = ['blue','green', 'red', 'orange']

In [ ]:
for season, color in zip(train.season.unique(), color_list):
    print(season, color)

In [ ]:

for season, c in zip(train.season.unique(), color_list):
    print(season)
    sns.jointplot(x= 'weekly_sales', y= 'temperature', data = train[train.season==season], color= c)
    plt.show()

### -  weekly_sales vs holiday_name

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(x= train['holiday_name'], y =train['weekly_sales'] )
plt.show()

**Takeaways**
- the 2 holidays with the greatest weekle_sales are pre_christmas and thanksgiving
-we can see christmas does not have large amount of weekly_sales it is because the people buy before christmas , it is not like blackfriday that the people actually buy in this specific week

### - weekly_sales vs season

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(x= train['season'], y =train['weekly_sales'] )

**takeaways**
- we see some outliers but basically there are the weekly sales for thanksgiving and christmas (pre)
- we can say summer has the lowest range of weekly sales amout

In [ ]:
for season, c in zip(train.season.unique(), color_list):
    print(season)
    sns.boxplot(x= train[train.season== season]['weekly_sales'], color = c )
    plt.show()

**takeaways**
- it looks like summer has the lowest median 

# model

In [18]:
#imports
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor 
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler


from sklearn.preprocessing import PolynomialFeatures

In [19]:

# #conver y_train y _ validate to df
y_train_df = pd.DataFrame( {'actual': y_train})
y_test_df = pd.DataFrame( {'actual': y_test})

In [20]:
def select_kbest  (X_df, y_df, n_features):
    '''
    Takes in the predictors, the target, and the number of features to select (k) ,
    and returns the names of the top k selected features based on the SelectKBest class
    
    X_df : the predictors
    y_df : the target
    n_features : the number of features to select (k)
    Example
    select_kbest(X_train_scaled, y_train, 2)
    '''
    
    f_selector = SelectKBest(score_func=f_regression, k= n_features)
    f_selector.fit(X_df, y_df)
    mask = f_selector.get_support()
    X_df.columns[mask]
    top = list(X_df.columns[mask])
    print(f'The top {n_features} selected feautures based on the SelectKBest class are: {top}' )
    return top

In [21]:
def select_rfe (X_df, y_df, n_features, method):
    '''
    Takes in the predictors, the target, and the number of features to select (k) ,
    and returns the names of the top k selected features based on the Recursive Feature Elimination (RFE)
    
    X_df : the predictors
    y_df : the target
    n_features : the number of features to select (k)
    method : LinearRegression, LassoLars, TweedieRegressor
    Example
    select_rfe(X_train_scaled, y_train, 2, LinearRegression())
    '''
    lm = method
    rfe = RFE(estimator=lm, n_features_to_select= n_features)
    rfe.fit(X_df, y_df)
    top_rfe = list(X_df.columns[rfe.support_])
    print(f'The top {n_features} selected feautures based on the the RFE class class are: {top_rfe}' )
    print(pd.Series(dict(zip(X_df.columns, rfe.ranking_))).sort_values())
    return top_rfe


In [22]:
def regression_errors(df, y, yhat):
    '''
    Takes in a dataframe , y = column with actual_values and yhat= name of the columns with predicted_values
    and calculate:
    sum of squared errors (SSE)
    explained sum of squares (ESS)
    total sum of squares (TSS)
    mean squared error (MSE)
    root mean squared error (RMSE)
    Returns a dictionary with all these values.
    Example:
    plot_residuals(df, 'tip', 'yhat')
    '''
    #import
    from sklearn.metrics import  mean_squared_error
    from math import sqrt
    
    
    #calculate SSE using sklearn
    SSE = mean_squared_error(df[y], df[yhat])*len(df)
    #explained sum of squares (ESS)
    ESS = ((df[yhat] - df[y].mean())**2).sum()
    #total sum of squares (TSS)
    TSS = ((df[y] - df[y].mean())**2).sum()
    #mean squared error (MSE)
    MSE = mean_squared_error(df[y], df[yhat])
    #root mean squared error (RMSE)
    RMSE = sqrt(MSE)
    
    #create a dictionary
    m= {
        'sse': SSE,
        'ess': ESS,
        'rmse': RMSE,
        'tss': TSS,
        'mse': MSE,
        'r2': ESS/TSS,
    }

    return m

In [23]:
def create_model (X_df_scaled, y_df, actual, method, name):
    '''
    takes in features scaled df, target df, name of actual target, 
    type of method and the name of the selected method and 
    returns a dictionary that contains calculated regression errors.
    
    X_df_scaled : df that contains scaled featues
    y_df: target df
    actual: name of the column where is actual value of the target
    mehod: type of method to create the model object
    name: enter the new name for your model
    
    Example:
    create_model(X_train_scaled[top_sb], y_train, 'actual', LinearRegression(normalize=True), 'modelOLS' )
    '''
    # fit the thing
    method.fit(X_df_scaled, y_df[actual])

    # predict train
    y_df[name] = method.predict(X_df_scaled)

    #calculate regression errors using a created function
    train_eval = regression_errors(y_df, actual, name)

    return train_eval

In [24]:
def report(metric_df):
    
    from IPython.display import display, HTML
    rmse_base = metric_df.iloc[0,2]
    print(f'These are the models that perform better than our baseline rmse: {rmse_base}')
    dfs =metric_df[['model', 'rmse_validate']][metric_df['rmse_validate'] < rmse_base]
    display(HTML(dfs.to_html()))
    
    
    min_val = metric_df['rmse_validate'].idxmin()
    metric_df.iloc[min_val][0]
    rsme_bet = round(metric_df['rmse_validate'].iloc[min_val], 2)
    print('-----------------------------------------------------------------------------------------------')
    print(f'   ********** The model with the less  rmse_validate  is {metric_df.iloc[min_val][0] }  rmse:{rsme_bet} **********             ')
    print('-----------------------------------------------------------------------------------------------')
    print(' ')
    min_val = metric_df['r^2_validate'].idxmax()
    metric_df.iloc[min_val][0]
    print(f'The model with r^2 validate closer to 1 is ', metric_df.iloc[min_val][0])
    
    display(HTML(metric_df.to_html()))
    return

In [25]:
def baseline_errors(df, y, option ):
    '''
    Takes in a dataframe , y = column with actual_values 
    and calculate:
    sum of squared errors (SSE)
    explained sum of squares (ESS)
    total sum of squares (TSS)
    mean squared error (MSE)
    root mean squared error (RMSE)
    Returns a dictionary with all these values
    Example:
    baseline_mean_errors(y_train, 'actual')
    '''
    #import
    from sklearn.metrics import  mean_squared_error
    from math import sqrt

    #baseline
    if option == 'mean':
        df['yhat_baseline_mean'] = df[y].mean()
        col = 'yhat_baseline_mean'
    elif option == 'median':
        df['yhat_baseline_median'] = df[y].median()
        col = 'yhat_baseline_median'
    else:
        return print("please select the correct option: 'mean' or 'median' ")
        
    

    #calculate SSE using sklearn
    SSE_baseline = mean_squared_error(df[y], df[col])*len(df)
    #explained sum of squares (ESS)
    ESS_b = ((df[col] - df[y].mean())**2).sum()
    #total sum of squares (TSS)
    TSS_b = ((df[y] - df[y].mean())**2).sum()
    #mean squared error (MSE)
    MSE_baseline = mean_squared_error(df[y], df[col])
    #root mean squared error (RMSE)
    RMSE_baseline = sqrt(MSE_baseline)
    
    #create dicc
    b ={
        'sse': SSE_baseline,
        'mse': MSE_baseline,
        'rmse': RMSE_baseline,
         'tss': TSS_b,
        'ess' : ESS_b,
        'mse': MSE_baseline,
        'r2': ESS_b/TSS_b,       
    }

    return b

**- Baseline**

In [26]:
y_train.mean()

1044296.2597557559

In [27]:
y_train.median()

957922.2949999999

In [28]:
#create baseline using mean (I'm using my function to calculate rmse)
tra = baseline_errors(y_train_df, 'actual', 'mean')
tra['rmse']

563052.7256939057

In [29]:
#create a df to store metrics
metric_df = pd.DataFrame(data = [{
    'model': 'mean_baseline',
    'rmse_train' : round(tra['rmse'], 2),
    'r^2' : 0}])

In [30]:
metric_df

,model,rmse_train,r^2
0,mean_baseline,563052.73,0


### - select k best

In [31]:
#dropping the columns that are related with our target
X_train_scaled = X_train_scaled.drop(columns = ['deflated_series','sales_delta_weekly','sales_delta_yearly'])

In [32]:
X_test_scaled =   X_test_scaled.drop(columns = ['deflated_series','sales_delta_weekly','sales_delta_yearly'])

In [33]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 6)

The top 6 selected feautures based on the SelectKBest class are: ['unemployment', 'store_size', 'last_year_sales', 'last_week_sales', 'pre_christmas', 'thanksgiving']


##  LinearRegression (OLS)

### - RFE

In [34]:
top_rfe = select_rfe(X_train_scaled, y_train, 8,LinearRegression(normalize=True, fit_intercept=True ))

The top 8 selected feautures based on the the RFE class class are: ['holiday_flag', 'unemployment', 'last_year_sales', 'last_week_sales', 'christmas', 'labor_day', 'super_bowl', 'thanksgiving']
holiday_flag         1
super_bowl           1
labor_day            1
christmas            1
last_week_sales      1
last_year_sales      1
thanksgiving         1
unemployment         1
week_of_year         2
quarter              3
pre_christmas        4
store_size           5
fuel_price           6
year                 7
gas_delta_yearly     8
temperature          9
inflation           10
gas_delta_weekly    11
tax_season          12
dtype: int64


### - cross validation

In [35]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

In [36]:
#create the model
clf = LinearRegression(normalize=True)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=5).mean()

0.9826501779236793

In [37]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [38]:
#create the model
clf = LinearRegression(normalize=True)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring = 'r2').mean()

0.9826501779236793

In [39]:
#create the model
clf = LinearRegression(normalize=True)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring = 'neg_root_mean_squared_error').mean()

-73910.79537108631

In [40]:
#create the model
clf = LinearRegression(normalize=False)
#cv = number of folds
cross_val_score(clf, X_train_scaled, y_train, cv=3).mean()

0.9825943012738833

## GridSearchCV

In [41]:
def gridsearch ( X_df , y_df , model, params, score):
    '''
    '''
    grid = GridSearchCV(model, params, cv=5, scoring= score )
    grid.fit(X_df, y_df)
    #see the cross validation results in the cv_results_ property of the object we created.
    results = grid.cv_results_
    # I will use mean_test_score
    test_scores = results['mean_test_score']
    #GETTING THE PARAMETERS
    params = results['params']
    #We can combine these features together into a data frame to see how our different models perform:
    for p, s in zip(params, test_scores):
        p['score'] = s

    return pd.DataFrame(params).sort_values(by='score')

In [42]:
from sklearn.model_selection import GridSearchCV

#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {'normalize': [ True, False],
          'fit_intercept': [True, False]}

ols = LinearRegression()

grid = GridSearchCV(ols, params, cv=5, scoring='neg_root_mean_squared_error')

grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False],
                         'normalize': [True, False]},
             scoring='neg_root_mean_squared_error')

In [43]:
#see the cross validation results in the cv_results_ property of the object we created.
results = grid.cv_results_
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_fit_intercept', 'param_normalize', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [44]:
# I will use mean_test_score
test_scores = results['mean_test_score']
test_scores

array([-73910.79537109, -73902.8390497 , -77238.39566647, -77238.39566647])

In [45]:
params = results['params']
params

[{'fit_intercept': True, 'normalize': True},
 {'fit_intercept': True, 'normalize': False},
 {'fit_intercept': False, 'normalize': True},
 {'fit_intercept': False, 'normalize': False}]

In [46]:
#We can combine these features together into a data frame to see how our different models perform:
for p, s in zip(params, test_scores):
    p['score'] = s

pd.DataFrame(params).sort_values(by='score')

,fit_intercept,normalize,score
2,False,True,-77238.395666
3,False,False,-77238.395666
0,True,True,-73910.795371
1,True,False,-73902.839050


In [47]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {'normalize': [ True, False],
          'fit_intercept': [True, False]}

In [49]:
gridsearch (X_train_scaled, y_train, LinearRegression() , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
2,False,True,-77238.395666
3,False,False,-77238.395666
0,True,True,-73910.795371
1,True,False,-73902.839050


In [50]:
gridsearch (X_train_scaled, y_train, LinearRegression() , params, 'r2')

,fit_intercept,normalize,score
2,False,True,0.981057
3,False,False,0.981057
0,True,True,0.982650
1,True,False,0.982654


In [51]:
### OLS uising  select K best (6)

In [52]:
ols_rfe = create_model(X_train_scaled[top_sb], y_train_df, 'actual', LinearRegression(normalize=True, fit_intercept=True ), 'modelOLS' )
ols_rfe['rmse']


75736.89572808747

In [53]:
### OLS uising  RFE (8)

In [54]:
ols_rfe = create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LinearRegression(normalize=True, fit_intercept=True ), 'modelOLS' )
ols_rfe['rmse']

74557.20308979187

In [55]:
metric_df = metric_df.append(
    {
    'model': 'ols_rfe',
    'rmse_train': ols_rfe['rmse'],    
    'r^2' : ols_rfe['r2']}, ignore_index=True)

In [56]:
metric_df

,model,rmse_train,r^2
0,mean_baseline,563052.73000,0.000000
1,ols_rfe,74557.20309,0.982213


## LassoLars

In [57]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
           'alpha': [1.0, 0]
         }

In [58]:
 LassoLars(alpha = .5)

LassoLars(alpha=0.5)

In [59]:
gridsearch (X_train_scaled, y_train, LassoLars(alpha = 1) , params, 'neg_root_mean_squared_error')

,alpha,fit_intercept,normalize,score
6,0.0,False,True,-77238.395666
7,0.0,False,False,-77238.395666
2,1.0,False,True,-77238.217188
3,1.0,False,False,-77238.217188
5,0.0,True,False,-73876.481581
4,0.0,True,True,-73876.481581
1,1.0,True,False,-73875.993658
0,1.0,True,True,-73873.466172


In [60]:
###RFE 

In [61]:
top_rfe = select_rfe(X_train_scaled, y_train, 8, LassoLars(alpha = 1, normalize= True, fit_intercept= True) )

The top 8 selected feautures based on the the RFE class class are: ['unemployment', 'quarter', 'week_of_year', 'last_year_sales', 'last_week_sales', 'christmas', 'pre_christmas', 'thanksgiving']
thanksgiving         1
pre_christmas        1
christmas            1
last_week_sales      1
unemployment         1
last_year_sales      1
quarter              1
week_of_year         1
store_size           2
holiday_flag         3
fuel_price           4
year                 5
gas_delta_yearly     6
temperature          7
inflation            8
super_bowl           9
gas_delta_weekly    10
tax_season          11
labor_day           12
dtype: int64


In [62]:
lasso_rfe = create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
lasso_rfe['rmse']

74486.97741108827

In [63]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 8)

The top 8 selected feautures based on the SelectKBest class are: ['temperature', 'unemployment', 'store_size', 'week_of_year', 'last_year_sales', 'last_week_sales', 'pre_christmas', 'thanksgiving']


In [64]:
lasso_skb = create_model(X_train_scaled[top_sb], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
lasso_skb['rmse']

75600.32914698498

In [65]:
lasso_rfe = create_model(X_train_scaled[top_rfe], y_train_df, 'actual', LassoLars(alpha = 1, normalize= False, fit_intercept= True), 'modelLasso' )
lasso_rfe['rmse']

74486.97741108827

In [66]:
metric_df = metric_df.append(
    {
    'model': 'lasso_rfe',
    'rmse_train': lasso_rfe['rmse'],    
    'r^2' : lasso_rfe['r2']}, ignore_index=True)

In [67]:
metric_df

,model,rmse_train,r^2
0,mean_baseline,563052.730000,0.000000
1,ols_rfe,74557.203090,0.982213
2,lasso_rfe,74486.977411,0.982473



## TweedieRegressor (GLM)

In [68]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'power': [0.0, 1],
           'fit_intercept' : [True , False],
          'warm_start': [True, False], 
           'alpha': [1.0, 0.0]
         }

In [69]:
gridsearch (X_train_scaled, y_train,TweedieRegressor() , params, 'neg_root_mean_squared_error')

,alpha,fit_intercept,power,warm_start,score
6,1.0,False,1.0,True,-1.937236e+06
7,1.0,False,1.0,False,-1.937236e+06
14,0.0,False,1.0,True,-1.937051e+06
15,0.0,False,1.0,False,-1.937051e+06
4,1.0,False,0.0,True,-5.505112e+05
5,1.0,False,0.0,False,-5.505112e+05
0,1.0,True,0.0,True,-5.012175e+05
1,1.0,True,0.0,False,-5.012175e+05
10,0.0,True,1.0,True,-1.467103e+05
11,0.0,True,1.0,False,-1.467103e+05


In [70]:
### RFE

In [71]:
top_rfe = select_rfe(X_train_scaled, y_train, 6, TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False) )

The top 6 selected feautures based on the the RFE class class are: ['unemployment', 'week_of_year', 'last_year_sales', 'last_week_sales', 'christmas', 'thanksgiving']
thanksgiving         1
christmas            1
unemployment         1
week_of_year         1
last_week_sales      1
last_year_sales      1
quarter              2
pre_christmas        3
store_size           4
fuel_price           5
year                 6
gas_delta_yearly     7
holiday_flag         8
temperature          9
inflation           10
super_bowl          11
gas_delta_weekly    12
labor_day           13
tax_season          14
dtype: int64


In [72]:
### select kbest

In [73]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 6)

The top 6 selected feautures based on the SelectKBest class are: ['unemployment', 'store_size', 'last_year_sales', 'last_week_sales', 'pre_christmas', 'thanksgiving']


In [74]:
gml_skb = create_model(X_train_scaled[top_sb], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
gml_skb['rmse']

75736.89572808956

In [75]:
gml_rfe = create_model(X_train_scaled[top_rfe], y_train_df, 'actual',TweedieRegressor(alpha =0 , fit_intercept= True, power=0 ,\
                                                                  warm_start= False), 'modelgml' )
gml_rfe['rmse']

74671.92773889692

In [76]:
metric_df = metric_df.append(
    {
    'model': 'gml_rfe',
    'rmse_train': gml_rfe['rmse'],    
    'r^2' : gml_rfe['r2']}, ignore_index=True)

In [77]:
metric_df

,model,rmse_train,r^2
0,mean_baseline,563052.730000,0.000000
1,ols_rfe,74557.203090,0.982213
2,lasso_rfe,74486.977411,0.982473
3,gml_rfe,74671.927739,0.982412


## plynomial Regression

In [78]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree = 3) 

# fit and transform X_train_scaled
X_train_degree3 = pf.fit_transform(X_train_scaled)

# transform X_validate_scaled & X_test_scaled
X_test_degree3 = pf.transform(X_test_scaled)

In [80]:
#specify the parameters we wish to use as a dictionary, then use that dictionary when we create the class.
params = {
          'normalize': [True, False],
          'fit_intercept':[True, False],
         }

In [81]:
gridsearch (X_train_degree3, y_train, LinearRegression(normalize=True) , params, 'neg_root_mean_squared_error')

,fit_intercept,normalize,score
0,True,True,-5.396751e+15
1,True,False,-2.032383e+14
2,False,True,-1.784725e+14
3,False,False,-1.784725e+14


In [85]:
gridsearch (X_train_degree3, y_train, LinearRegression(normalize=True) , params, 'r2')

,fit_intercept,normalize,score
0,True,True,-3.166867e+20
1,True,False,-1.796322e+17
2,False,True,-1.229404e+17
3,False,False,-1.229404e+17


In [ ]:
# select k best 

In [84]:
#using my function for SelectkBest
top_sb =select_kbest(X_train_scaled, y_train, 6)

The top 6 selected feautures based on the SelectKBest class are: ['unemployment', 'store_size', 'last_year_sales', 'last_week_sales', 'pre_christmas', 'thanksgiving']


In [ ]:
# RFE

In [92]:
 top_rfe = select_rfe(X_train_scaled, y_train, 6, LinearRegression(normalize=False, fit_intercept = False ) )

The top 6 selected feautures based on the the RFE class class are: ['temperature', 'gas_delta_weekly', 'last_year_sales', 'last_week_sales', 'christmas', 'thanksgiving']
gas_delta_weekly     1
christmas            1
last_week_sales      1
last_year_sales      1
thanksgiving         1
temperature          1
pre_christmas        2
gas_delta_yearly     3
year                 4
fuel_price           5
week_of_year         6
super_bowl           7
store_size           8
labor_day            9
holiday_flag        10
tax_season          11
inflation           12
quarter             13
unemployment        14
dtype: int64


In [97]:
pol_rfe = create_model(X_train_degree3, y_train_df, 'actual',LinearRegression( fit_intercept = False ), 'modelpol' )
pol_rfe['rmse']

41917.93455396443

In [98]:
metric_df = metric_df.append(
    {
    'model': 'pol_rfe',
    'rmse_train': pol_rfe['rmse'],    
    'r^2' : pol_rfe['r2']}, ignore_index=True)

In [100]:
metric_df

,model,rmse_train,r^2
0,mean_baseline,563052.730000,0.000000
1,ols_rfe,74557.203090,0.982213
2,lasso_rfe,74486.977411,0.982473
3,gml_rfe,74671.927739,0.982412
4,pol_rfe,41917.934554,0.992287
